In [2]:
%load_ext autoreload
%autoreload 2

In [13]:
%matplotlib inline
import pandas as pd
import numpy as np

import os

# The Other Side of Gradescope

The file contains the grade-book from a fictional data science course with 535 students. 

**Note: this dataset is synthetically generated; it does not contain real student grades.**

In this project, I will:
1. clean and process the data to compute total course grades according to a fictional syllabus (below),
2. qualitatively understand how students did in the course,
3. understand how student grades vary with small changes in performance on each assignment.

---

The course syllabus is as follows:

* Lab assignments 
    - Each are worth the same amount, regardless of each lab's raw point total.
    - The lowest lab is dropped.
    - Each lab may be revised for one week after submission for a 10% penalty, for two weeks after submission for a 20% penalty, and beyond that for a 50% penalty. Such revisions are reflected in the `Lateness` columns in the gradebook.
    - Labs are 20% of the total grade.
* Projects 
    - Each project consists of an autograded portion, and *possibly* a free response portion.
    - The total points for a single project consist of the sum of the raw score of the two portions.
    - Each are worth the same amount, regardless of each project's raw point total.
    - Projects are 30% of the total grade.
* Checkpoints
    - Project checkpoints are worth 2.5% of the total grade.
* Discussion
    - Discussion notebooks are worth 2.5% of the total grade.
* Exams
    - The midterm is worth 15% of the total grade.
    - The final is worth 30% of the total grade.


### A note on 'putting everything together'

The goal of this project is to create and assess final grades for a fictional course.

In [14]:
grades_fp = os.path.join('data', 'grades.csv')
grades = pd.read_csv(grades_fp)

In [15]:
grades.head()

,PID,College,Level,lab01,lab01 - Max Points,lab01 - Lateness (H:M:S),lab02,lab02 - Max Points,lab02 - Lateness (H:M:S),project01,...,discussion07 - Lateness (H:M:S),discussion08,discussion08 - Max Points,discussion08 - Lateness (H:M:S),discussion09,discussion09 - Max Points,discussion09 - Lateness (H:M:S),discussion10,discussion10 - Max Points,discussion10 - Lateness (H:M:S)
0,A14721419,SI,JR,99.0,100.0,00:00:00,86.0,100.0,00:00:00,75.0,...,00:00:00,10.0,10,00:00:00,10.0,10,780:01:28,10.0,10,00:00:00
1,A14883274,TH,JR,98.0,100.0,00:00:00,52.0,100.0,00:00:00,53.0,...,669:12:21,7.0,10,00:00:00,7.0,10,00:00:00,8.0,10,00:00:00
2,A14164800,SI,SR,86.0,100.0,00:00:00,45.0,100.0,00:00:00,44.0,...,00:00:00,6.0,10,00:04:51,6.0,10,00:00:00,7.0,10,00:00:00
3,A14847419,TH,JR,100.0,100.0,00:00:00,100.0,100.0,00:00:00,78.0,...,00:00:00,10.0,10,00:00:00,10.0,10,00:00:00,10.0,10,00:00:00
4,A14162943,SI,JR,66.0,100.0,00:00:00,33.0,100.0,00:00:00,42.0,...,00:00:00,5.0,10,00:00:00,5.0,10,00:00:00,6.0,10,00:00:00


In [16]:
grades[[ 'project02',
       'project02 - Max Points', 'project02 - Lateness (H:M:S)',
       'project02_free_response', 'project02_free_response - Max Points',
       'project02_free_response - Lateness (H:M:S)']].head()

,project02,project02 - Max Points,project02 - Lateness (H:M:S),project02_free_response,project02_free_response - Max Points,project02_free_response - Lateness (H:M:S)
0,75.0,75.0,00:00:00,18.0,25.0,00:00:00
1,64.0,75.0,00:00:00,25.0,25.0,00:00:00
2,63.0,75.0,00:00:00,25.0,25.0,00:00:00
3,69.0,75.0,00:00:00,25.0,25.0,00:00:00
4,71.0,75.0,00:00:00,24.0,25.0,00:00:00


### Getting started: enumerating the assignments

First, I will list all the 'assignment names' and what part of the syllabus to which they belong.

**Step 1:**

Create a function `get_assignment_names` that takes in a dataframe like `grades` and returns a dictionary with the following structure:
- The keys are the general areas of the syllabus: `lab, project, midterm, final, disc, checkpoint`
- The values are lists that contain the assignment names of that type. For example the lab assignments all have names of the form `labXX` where `XX` is a zero-padded two digit number. See the doctests for more details.

In [17]:
def get_assignment_names(grades):
    '''
    get_assignment_names takes in a dataframe like grades and returns 
    a dictionary with the following structure:

    The keys are the general areas of the syllabus: lab, project, 
    midterm, final, disc, checkpoint

    The values are lists that contain the assignment names of that type. 
    For example the lab assignments all have names of the form labXX where XX 
    is a zero-padded two digit number. 
    '''
    general = ['lab', 'project', 'midterm', 'final', 'disc', 'checkpoint']
    names = {i:[] for i in general}
    columns = grades.columns
    for i in columns:
        if 'lab' in i.lower() and len(i) <= len('lab') + 2:
            names['lab'].append(i)
        elif 'midterm' in i.lower() and len(i) <= len('midterm') + 2:
            names['midterm'].append(i)
        elif 'final' in i.lower() and len(i) <= len('final') + 2:
            names['final'].append(i)
        elif 'disc' in i.lower() and len(i) <= len('discussion') + 2:
            names['disc'].append(i)
        elif 'project' in i.lower():
            if len(i) <= len('project') + 2:
                names['project'].append(i)
            elif 'checkpoint' in i[10:]and len(i) <= 23:
                names['checkpoint'].append(i)
    return names

In [18]:
names = get_assignment_names(grades)
names

{'lab': ['lab01',
  'lab02',
  'lab03',
  'lab04',
  'lab05',
  'lab06',
  'lab07',
  'lab08',
  'lab09'],
 'project': ['project01', 'project02', 'project03', 'project04', 'project05'],
 'midterm': ['Midterm'],
 'final': ['Final'],
 'disc': ['discussion01',
  'discussion02',
  'discussion03',
  'discussion04',
  'discussion05',
  'discussion06',
  'discussion07',
  'discussion08',
  'discussion09',
  'discussion10'],
 'checkpoint': ['project02_checkpoint01',
  'project02_checkpoint02',
  'project03_checkpoint01']}

### Computing project grades

**Step 2**

Compute the total score for the project portion of the course according to the syllabus. Create a function `projects_total` that takes in `grades` and computes the total project grade for the quarter according to the syllabus. The output Series contains values between 0 and 1.


In [19]:
def projects_total(grades):
    '''
    projects_total that takes in grades and computes the total project grade
    for the quarter according to the syllabus. 
    The output Series should contain values between 0 and 1.
    
    Projects 
    Each project consists of an autograded portion, and possibly a free response portion.
    The total points for a single project consist of the sum of the raw score of the two portions.
    Each are worth the same amount, regardless of each project's raw point total.
    Projects are 30% of the total grade.
    '''
    project_cols = [col for col in grades.columns if 'project' in col]
    project = grades[project_cols]
    project_num = ['project01', 'project02', 'project03', 'project04', 'project05']
    rows = len(grades.index)
    project_grades = pd.Series([0 for i in range(rows)])
    for i in project_num:
        one_project_cols = [col for col in project.columns if i in col]
        one_project =  project[one_project_cols]
        if i + '_free_response' in one_project_cols:
            actual_grade = (one_project[i] + one_project[i + '_free_response'])
            max_grade = (one_project[i + ' - Max Points'] + one_project[i + '_free_response - Max Points'])
            grade = actual_grade/max_grade
            grade = grade.reset_index(drop = True)
            project_grades = project_grades.add(grade, fill_value=0)
        else:
            grade = one_project[i]/one_project[i + ' - Max Points']
            grade = grade.reset_index(drop = True)
            project_grades = project_grades.add(grade, fill_value=0)
    project_grades = project_grades / 5
    return project_grades

In [20]:
out = projects_total(grades)
out

0      0.900000
1      0.759333
2      0.673333
3      0.952667
4      0.718667
5      0.908000
6      0.779333
7      0.931333
8      0.852667
9      0.918000
10     0.920000
11     0.695333
12     0.955333
13     0.922667
14     0.790000
15     0.876667
16     0.982000
17     0.568000
18     0.946000
19     0.884667
20     0.865333
21     0.841333
22     0.863333
23     0.936000
24     0.887333
25     0.824667
26     0.854000
27     0.844000
28     0.825333
29     0.712667
         ...   
505    0.947333
506    0.913333
507    0.781333
508    0.844000
509    0.949333
510    0.917333
511    0.576667
512    0.970000
513    0.654000
514    0.944000
515    0.798667
516    0.787333
517    0.746000
518    0.880667
519    0.940000
520    0.829333
521    0.980000
522    0.934667
523    0.868000
524    0.912667
525    0.865333
526    0.920667
527    0.953333
528    0.930667
529    0.741333
530    0.949333
531    0.846667
532    0.837333
533    0.797333
534    0.948000
Length: 535, dtype: floa

### Computing lab grades

Now, I will clean and process the lab grades, which is a little more complicated. To do this, I will develop functions that:
- 'normalize' the grades, 
- adjust for late submissions, 
- drop the lowest lab grade, and 
- creates a total lab score for each student.

**Step 3**

Unfortunately, Gradescope sometimes experiences a delay in registering when an assignment is submitted during "periods of heavy usage" (i.e. near a submission deadline). I need to assess when a student's assignment was actually turned in on time, even if Gradescope did not process it in time. Note:
* Every late submission has to be submitted by a TA (late submissions are turned off).
* TAs never submitted a late assignment "just after" the deadline. 
* The deadlines were at midnight and students had to come to staff hours to late-submit their assignment.

Create a function `last_minute_submissions` that takes in the dataframe `grades` and outputs the number of submissions on each assignment that were turned in on time by the student, yet marked 'late' by Gradescope. 

In [21]:
from matplotlib import pyplot as plt 

In [24]:
def last_minute_submissions(grades):
    """
    last_minute_submissions takes in the dataframe 
    grades and a Series indexed by lab assignment that 
    contains the number of submissions that were turned 
    in on time by the student, yet marked 'late' by Gradescope.

    """
    result = pd.Series()
    for d in range(1,10):    
        grades[['hh','mm', 'ss']] = grades['lab0%d - Lateness (H:M:S)' % d].str.split(":",expand=True)
        hour = grades['hh'].astype(np.int)
        second = grades['ss'].astype(np.int)
        false_late = (((second.apply(lambda x: x) > 0) & (hour.apply(lambda x: x) <= 8)).astype(int)).sum()
        result['lab0%d' % d] = false_late
    return result

In [27]:
out = last_minute_submissions(grades)
out

lab01     0
lab02     0
lab03     0
lab04    12
lab05     0
lab06     0
lab07     0
lab08     0
lab09     0
dtype: int64

**Step 4**

Now I need to adjust the lab grades for late submissions -- however, I need to take into account my investigation in the previous question, since students shouldn't be penalized by a bug in Gradescope!

Create a function `lateness_penalty` that takes in a 'Lateness' column and returns a column of penalties (represented by the values `1.0,0.9,0.8,0.5` according to the syllabus). Only *truly* late submissions should be counted as late.

In [28]:
def lateness_penalty(col):
    """
    adjust_lateness takes in the dataframe like `grades` 
    and returns a dataframe of lab grades adjusted for 
    lateness according to the syllabus.
    
    Each are worth the same amount, regardless of each lab's raw point total.
    The lowest lab is dropped.
    Each lab may be revised for one week after submission for a 10% penalty,
    for two weeks after submission for a 20% penalty, and beyond that for a 50% penalty. 
    Such revisions are reflected in the Lateness columns in the gradebook.
    Labs are 20% of the total grade.

    :Example:
    >>> fp = os.path.join('data', 'grades.csv')
    >>> col = pd.read_csv(fp)['lab01 - Lateness (H:M:S)']
    >>> out = lateness_penalty(col)
    >>> isinstance(out, pd.Series)
    True
    >>> set(out.unique()) <= {1.0, 0.9, 0.8, 0.5}
    True
    """
        
    out = col.str.split(':').apply(lambda x: x[0]).astype(int) # Get hour number
    out = out.apply(lambda x: 1 if x < 8 else x)\
    .apply(lambda x: 0.9 if x >= 8 and x < 168 else x)\
    .apply(lambda x: 0.8 if x >= 168 and x < 336 else x)\
    .apply(lambda x: 0.5 if x >= 336 else x)
    return out

In [29]:
fp = os.path.join('data', 'grades.csv')
col = pd.read_csv(fp)['lab01 - Lateness (H:M:S)']
out = lateness_penalty(col)
out

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
5      1.0
6      1.0
7      1.0
8      1.0
9      1.0
10     1.0
11     1.0
12     1.0
13     1.0
14     1.0
15     1.0
16     1.0
17     1.0
18     1.0
19     1.0
20     0.9
21     1.0
22     1.0
23     1.0
24     1.0
25     1.0
26     1.0
27     1.0
28     1.0
29     1.0
      ... 
505    1.0
506    0.9
507    1.0
508    1.0
509    1.0
510    1.0
511    1.0
512    1.0
513    1.0
514    0.9
515    1.0
516    1.0
517    1.0
518    1.0
519    1.0
520    1.0
521    1.0
522    1.0
523    1.0
524    1.0
525    1.0
526    1.0
527    1.0
528    1.0
529    1.0
530    0.9
531    1.0
532    1.0
533    1.0
534    1.0
Name: lab01 - Lateness (H:M:S), Length: 535, dtype: float64

**Step 5**

Create a function `process_labs` that takes in a dataframe like `grades` and returns a dataframe of processed lab scores. The output should:
* share the same index as `grades`,
* have columns given by the lab assignment names (e.g. `lab01,...lab10`)
* have values representing the lab grades for each assignment, adjusted for Lateness and scaled to a score between 0 and 1.

Lab assignments 
Each are worth the same amount, regardless of each lab's raw point total.
The lowest lab is dropped.
Each lab may be revised for one week after submission for a 10% penalty, for two weeks after submission for a 20% penalty, and beyond that for a 50% penalty. Such revisions are reflected in the Lateness columns in the gradebook.
Labs are 20% of the total grade.

In [30]:
def process_labs(grades):
    """
    process_labs that takes in a dataframe like grades and returns
    a dataframe of processed lab scores. The output should:
      * share the same index as grades,
      * have columns given by the lab assignment names (e.g. lab01,...lab10)
      * have values representing the lab grades for each assignment, 
        adjusted for Lateness and scaled to a score between 0 and 1.

    """
    lab = grades
    for d in range(1,10):  
        penulty = lateness_penalty(lab['lab0%d - Lateness (H:M:S)' % d])
        lab['lab0%d' % d] = lab['lab0%d' % d] / lab['lab0%d - Max Points' % d]
        lab['lab0%d' % d] = lab['lab0%d' % d] * penulty
    return lab[['lab%02d' % x for x in range(1,10)]]

In [31]:
fp = os.path.join('data', 'grades.csv')
grades = pd.read_csv(fp)
out = process_labs(grades)
out

,lab01,lab02,lab03,lab04,lab05,lab06,lab07,lab08,lab09
0,0.990,0.860,0.720,0.980,1.000000,0.976471,0.485,0.880,0.860
1,0.980,0.520,0.730,0.770,1.000000,0.500000,0.890,0.940,0.860
2,0.860,0.450,0.400,0.730,0.900000,0.429412,0.720,0.710,0.760
3,1.000,1.000,0.920,0.910,0.885714,0.670588,1.000,0.950,0.780
4,0.660,0.330,0.690,0.729,0.642857,0.741176,0.600,0.360,1.000
5,0.910,1.000,1.000,0.970,1.000000,0.917647,0.910,1.000,0.860
6,0.960,0.470,0.740,0.756,0.871429,0.752941,0.490,0.640,0.540
7,1.000,0.980,0.970,1.000,0.971429,0.952941,0.910,0.920,0.980
8,0.900,1.000,0.990,0.873,1.000000,0.476471,0.860,0.620,1.000
9,1.000,1.000,0.820,0.801,0.900000,0.458824,0.920,0.500,0.740


**Step 6**

Create a function `lab_total` that takes in dataframe of processed assignments (like the output of Question 5) and computes the total lab grade for each student according to the syllabus (returning a Series). The answers should be proportions between 0 and 1. For example, if there are only 3 labs, and a student received scores of {80%,90%,100%}, then the total score would be 0.95.



In [32]:
def lab_total(processed):
    """
    lab_total takes in dataframe of processed assignments (like the output of 
    Question 5) and computes the total lab grade for each student according to
    the syllabus (returning a Series). 
    
    The answers should be proportions between 0 and 1.
    """

    out = processed.fillna(0)
    
    return (out.sum(axis=1) - out.min(axis=1))/(out.shape[1] - 1)

In [33]:
grades.columns

Index(['PID', 'College', 'Level', 'lab01', 'lab01 - Max Points',
       'lab01 - Lateness (H:M:S)', 'lab02', 'lab02 - Max Points',
       'lab02 - Lateness (H:M:S)', 'project01', 'project01 - Max Points',
       'project01 - Lateness (H:M:S)', 'lab03', 'lab03 - Max Points',
       'lab03 - Lateness (H:M:S)', 'project01_free_response',
       'project01_free_response - Max Points',
       'project01_free_response - Lateness (H:M:S)', 'lab04',
       'lab04 - Max Points', 'lab04 - Lateness (H:M:S)', 'lab05',
       'lab05 - Max Points', 'lab05 - Lateness (H:M:S)',
       'project02_checkpoint01', 'project02_checkpoint01 - Max Points',
       'project02_checkpoint01 - Lateness (H:M:S)', 'Midterm',
       'Midterm - Max Points', 'Midterm - Lateness (H:M:S)', 'lab06',
       'lab06 - Max Points', 'lab06 - Lateness (H:M:S)',
       'project02_checkpoint02', 'project02_checkpoint02 - Max Points',
       'project02_checkpoint02 - Lateness (H:M:S)', 'lab07',
       'lab07 - Max Points', 'lab07 

In [34]:
fp = os.path.join('data', 'grades.csv')
grades = pd.read_csv(fp)
lab_total(grades[['lab01', 'lab02', 'lab03', 'lab04', 'lab05', 'lab06', 'lab07', 'lab08', 'lab09']])

0      88.875
1      79.750
2      65.375
3      87.125
4      58.750
5      90.875
6      67.375
7      90.875
8      85.625
9      81.750
10     88.500
11     65.625
12     84.250
13     87.125
14     85.125
15     79.625
16     89.500
17     54.000
18     86.875
19     80.750
20     75.500
21     82.250
22     65.500
23     78.125
24     75.000
25     78.375
26     74.375
27     78.500
28     77.375
29     75.125
        ...  
505    84.625
506    84.250
507    55.500
508    84.250
509    91.625
510    82.125
511    70.000
512    80.875
513    66.750
514    79.750
515    75.875
516    80.125
517    79.125
518    90.625
519    83.250
520    92.500
521    87.375
522    85.750
523    73.500
524    77.000
525    86.125
526    78.500
527    91.375
528    83.000
529    67.625
530    83.250
531    78.125
532    86.000
533    81.250
534    80.875
Length: 535, dtype: float64

### Putting it together

**Step 7**

Finally, I need to create the final course grades. To do this, I will add up the total of each course component according to the weights given in the syllabus. 

* Create a function `total_points` that takes in `grades` and returns the final course grades according to the syllabus. Course grades should be proportions between zero and one.
* Create a function `final_grades` that takes in the final course grades as above and returns a Series of letter grades given by the standard cutoffs (`A >= .90`, `.90 > B >= .80`, `.80 > C >= .70`, `.70 > D >= .60`, `.60 > F`). 
* Create a function `letter_proportions` which takes in the dataframe `grades` and outputs a Series that contains the proportion of the class that received each grade. (This question requires me to put everything together).



In [35]:
def letter_proportions(grades):
    """
    letter_proportions takes in the dataframe grades 
    and outputs a Series that contains the proportion
    of the class that received each grade.

    :Example:
    >>> fp = os.path.join('data', 'grades.csv')
    >>> grades = pd.read_csv(fp)
    >>> out = letter_proportions(grades)
    >>> np.all(out.index == ['B', 'C', 'A', 'D', 'F'])
    True
    >>> out.sum() == 1.0
    True
    """

    return final_grades(total_points(grades)).value_counts(normalize=True) 

In [38]:
fp = os.path.join('data', 'grades.csv')
grades = pd.read_csv(fp)
out = letter_proportions(grades)
out

C:\Users\lucyg\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


B    0.528972
C    0.248598
A    0.140187
D    0.048598
F    0.033645
dtype: float64

In [39]:
def letter_grade(grade):
    if grade >= 0.9:
        return 'A'
    elif grade < 0.9 and grade >= 0.8:
        return 'B'
    elif grade < 0.8 and grade >= 0.7:
        return 'C'
    elif grade < 0.7 and grade >= 0.6:
        return 'D'
    else:
        return 'F'

In [40]:
def final_grades(total):
    """
    final_grades takes in the final course grades
    as above and returns a Series of letter grades
    given by the standard cutoffs.

    :Example:
    >>> out = final_grades(pd.Series([0.92, 0.81, 0.41]))
    >>> np.all(out == ['A', 'B', 'F'])
    True
    """

    return total.apply(letter_grade)


In [41]:
out = final_grades(pd.Series([0.92, 0.81, 0.41]))
out

0    A
1    B
2    F
dtype: object

In [37]:
def total_points(grades):
    """
    total_points takes in grades and returns the final
    course grades according to the syllabus. Course grades
    should be proportions between zero and one.

    """

    grades_temp = grades.fillna(0).copy()
    
#     Labs are 20% of the total grade.
    lab = lab_total(process_labs(grades_temp))*0.2


#     Projects are 30% of the total grade.
    project = projects_total(grades_temp)*0.3

#     Project checkpoints are worth 2.5% of the total grade.
    check = grades_temp[grades_temp.columns[grades_temp.columns.str.contains('checkpoint')]]
    num_checks = int(check.shape[1]/3)
    for d in range(num_checks):
        check['check%d' % d] = check.iloc[:, d*3]/check.iloc[:, d*3 + 1]
    checkpoint = check[['check%d' % d for d in range(num_checks)]].mean(axis = 1)*0.025 

#     Discussion notebooks are worth 2.5% of the total grade.
    disc = grades_temp
    for d in range(1,11):
        if d >= 10:
            disc['discussion0%d' % d] = disc['discussion10'] / disc['discussion%d - Max Points' % d]
        else:
            disc['discussion0%d' % d] = disc['discussion0%d' % d] / disc['discussion0%d - Max Points' % d]
    discussion = disc[['discussion0%d' % x for x in range(1,11)]].mean(axis = 1)*0.025

#     The midterm is worth 15% of the total grade.
    midterm = (grades_temp['Midterm']/grades_temp['Midterm - Max Points'])*0.15


#     The final is worth 30% of the total grade.  
    final = (grades_temp['Final']/grades_temp['Final - Max Points'])*0.3
#     grade = np.clip(labs, 0, 1)
    return np.clip(lab + project + checkpoint + discussion + midterm + final, 0, 1)

In [59]:
total_points(grades)

C:\Users\lucyg\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0      0.887323
1      0.804042
2      0.742045
3      0.900989
4      0.648927
5      0.902117
6      0.794718
7      0.948493
8      0.863896
9      0.879548
10     0.895998
11     0.703816
12     0.880309
13     0.941230
14     0.810327
15     0.882163
16     0.912671
17     0.657350
18     0.918531
19     0.859532
20     0.778686
21     0.844022
22     0.802340
23     0.875860
24     0.813029
25     0.719387
26     0.817281
27     0.814022
28     0.871529
29     0.770708
         ...   
505    0.862301
506    0.857249
507    0.666339
508    0.802477
509    0.966685
510    0.827225
511    0.696905
512    0.924085
513    0.717919
514    0.888544
515    0.835430
516    0.807867
517    0.705774
518    0.902604
519    0.890388
520    0.811068
521    0.932655
522    0.915092
523    0.776675
524    0.769602
525    0.850429
526    0.841147
527    0.917945
528    0.815424
529    0.780180
530    0.861339
531    0.749270
532    0.843537
533    0.851361
534    0.897919
Length: 535, dtype: floa